In [14]:
sc


In [15]:
orderItems = sc.textFile("sqoop_import/retail_db/order_items")

In [21]:
print(orderItems.take(1))

orderItemsMap = orderItems.map( lambda oi: (float(oi.split(",")[1]) ,  float(oi.split(",")[4])))

[u'1,1,957,1,299.98,299.98']


orderItemsMap = orderItems.map( lambda a: a.split(",")[4])

In [17]:
reducedOrderItems = orderItemsMap.reduceByKey(lambda curr , next: curr + next)


In [24]:
for s in reducedOrderItems.take(1): print(s)

(32768.0, 1029.94)


In [63]:
l = range(1,10000)
lRDD = sc.parallelize(l)
type(lRDD)

pyspark.rdd.RDD

In [ ]:
productsRaw = open("/home/zubairidrees/hadoop-2.7.3/data/retail_db/products/part-00000").read().splitlines()
print type(productsRaw)
productsRDD = sc.parallelize(productsRaw)
productsRDD.take(1)

In [200]:
sqlContext
help(sqlContext)

In [199]:
#spark 1.6
#sqlContext.load("/retail_db_json/order_items","json").show()

#spark 2
spark.read.load("/user/zubairidrees/retail_db/json/retail_db_json/order_items", format="json").show()

#sqlContext.read.json("/user/zubairidrees/retail_db/json/retail_db_json/order_items/part-r-00000-6b83977e-3f20-404b-9b5f-29376ab1419e").show()
spark.read.json("/user/zubairidrees/retail_db/json/retail_db_json/order_items").show()


## String Manipulation


In [87]:
orders = sc.textFile("retail_db/retail_db/orders")
orders.take(1)

[u'1,2013-07-25 00:00:00.0,11599,CLOSED']

In [102]:
s = orders.first()
type(s)
s[2:13]
len(s)
print("Concat String with int" + str(1))




Concat String with int1


u'20130725'

## Convert Date YYYY-MM-DD into YYYYMMDD



In [103]:
s.split(",")[1].split(" ")[0].replace("-","")

u'20130725'

## map 

In [203]:
orders = sc.textFile("retail_db/retail_db/orders")
orders.map(lambda o: o.split(",")[1].split(" ")[0].replace("-","")).take(10)
orders.map(lambda o: o.split(",")[1].split(" ")[0].replace("-","")).count()

orderItems = sc.textFile("retail_db/retail_db/order_items")
orderItems.first()
orderItems.map(lambda oi:( int(oi.split(",")[1]),float(oi.split(",")[5]) )).take(1)


[(1, 299.98)]

## flatMap

In [136]:
linesList = ["How are you","where are you","How is the things going"]
linesRDD = sc.parallelize(linesList)

linesMap = linesRDD.map(lambda l :l.split(" ")).collect()
for i in linesMap: print i

linesFlatMap = linesRDD.flatMap(lambda l :l.split(" ")).collect()
for i in linesFlatMap: print i



['How', 'are', 'you']
['where', 'are', 'you']
['How', 'is', 'the', 'things', 'going']
How
are
you
where
are
you
How
is
the
things
going


## filter

In [204]:
orders = sc.textFile("retail_db/retail_db/orders")

ordersCompleted = orders.filter(lambda o: o.split(",")[3] == "COMPLETE" or o.split(",")[3] == "CLOSED").take(1)
for i in ordersCompleted: print i
    
    
#filter records of year 2014

orders2014 = orders.filter(lambda o: 
                           (o.split(",")[3] in [ "COMPLETE", "CLOSED"]) and
                           o.split(",")[1].split("-")[0] == "2014").take(1)
for i in orders2014: print i


1,2013-07-25 00:00:00.0,11599,CLOSED
25882,2014-01-01 00:00:00.0,4598,COMPLETE


## Join

In [179]:
orders = sc.textFile("retail_db/retail_db/orders")
orderItems = sc.textFile("retail_db/retail_db/order_items")

ordersMap = orders.map(lambda o: (int(o.split(",")[0]) ,o.split(",")[3] ))

orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]) ,float(oi.split(",")[4]) ))

ordersJoin = ordersMap.join(orderItemsMap)


ordersLeftoin = ordersMap.leftOuterJoin(orderItemsMap)

orderOuterLeftJoinFilter = ordersLeftoin.filter(lambda o: o[1][1] == None)
print orderOuterLeftJoinFilter.count()
for i in orderOuterLeftJoinFilter.take(1): print i


11452
(43692, (u'PENDING_PAYMENT', None))


## Reduce

In [198]:
orderItems = sc.textFile("retail_db/retail_db/order_items")
orderItemsFiltered = orderItems.filter(lambda oi: int(oi.split(",")[1]) == 2)


orderItemsMap = orderItemsFiltered.map(lambda oi: float(oi.split(",")[4])) 
#for i in orderItemsMap.take(3): print i  
orderItemsReduced = orderItemsMap.reduce(lambda curr , next : curr + next)
print orderItemsReduced

## using operator
from operator import add
orderItemsReduced = orderItemsMap.reduce(add)
print orderItemsReduced

##using if else in reduce
##Find provided order id order item detail having minimum sub total

orderItemsReduced = orderItemsMap.reduce(lambda curr , next : next if(curr > next) else curr)
print orderItemsReduced

 


579.98
579.98
129.99


## ReduceByKey / CountByKey / AggregateByKey

# Count By Status

In [210]:
orders = sc.textFile("retail_db/retail_db/orders")
for i in orders.take(1):print i
ordersStatusMap = orders.map(lambda o: (o.split(",")[3], 1) )
for i in ordersStatusMap.take(1): print i
    
orderStatusCount = ordersStatusMap.countByKey();
print orderStatusCount


1,2013-07-25 00:00:00.0,11599,CLOSED
(u'CLOSED', 1)
defaultdict(<type 'int'>, {u'COMPLETE': 22899, u'PAYMENT_REVIEW': 729, u'PROCESSING': 8275, u'CANCELED': 1428, u'PENDING': 7610, u'CLOSED': 7556, u'PENDING_PAYMENT': 15030, u'SUSPECTED_FRAUD': 1558, u'ON_HOLD': 3798})


#### groupByKey don't use combiner & thats why iit is slow compare to reduceByKey & aggregateByKey
#### groupByKey
#### sum of (1-1000) -> 1 + 2 + 3 + 4 + 5 + 6 ......

#### reduceByKey & aggregateByKey
#### sum of (1-1000) -> sum(1,25) + sum(26,50)...

#### in case of average we can't use reduceByKey & aggregateByKey in place of groupByKey as result will be
#### different so if result of reduceByKey/aggregateByKey is same then only use it

#### reduceByKey take one function & aggregateByKey take 2 functions one is for computing the intermidiate value & other for final value

#### if reducer & combinor  is same then use reduceByKey




#Get Revenue for each order_id


orderItems = sc.textFile("retail_db/retail_db/order_items")
#for i in orderItems.take(1):print i

orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]), float(oi.split(",")[4])) )
for i in orderItemsMap.take(2):print i
    
orderItemsGroupByKey = orderItemsMap.groupByKey();
#print orderItemsGroupByKey.first()[1]

print "group by"

for i in orderItemsGroupByKey.take(2): 
    orderItemsGroupByKey

revenueByOrderIdMap = orderItemsGroupByKey.map(lambda oi: ( oi[0], sum(oi[1]) ) )

for i in revenueByOrderIdMap.collect():
    if(i[0] == 7):
        print i
    

In [1]:
cost = [['Lounge', 70], ['Kitchen', 70]]
print("£", sum(c[1] for c in cost))


('\xc2\xa3', 140)


In [2]:
###Get order item detail for each order id in decending order by revenue
orderItems = sc.textFile("retail_db/retail_db/order_items")
print orderItems.first()
orderItemsMap = orderItems.map(lambda oi: (oi.split(",")[1], oi))

    
orderItemsGroupByOrderId = orderItemsMap.groupByKey()
print "order items group by key"
for i in orderItemsGroupByOrderId.take(3): 
    print (i[0])
    g = list(i[1])
    print g

#l = orderItemsGroupByOrderId.first();
#list(l[1])

orderItemSortedBySubtotalPerOrderId = orderItemsGroupByOrderId.flatMap \
(lambda oi:sorted(oi[1],key=lambda k: float(k.split(",")[4]),reverse=True) ) 
#help(sorted)

print "sorted order item group by key"
for i in orderItemSortedBySubtotalPerOrderId.take(10): print i



1,1,957,1,299.98,299.98
order items group by key
11542
[u'28860,11542,1014,1,49.98,49.98', u'28861,11542,365,4,239.96,59.99', u'28862,11542,627,2,79.98,39.99', u'28863,11542,191,5,499.95,99.99', u'28864,11542,1014,3,149.94,49.98']
11540
[u'28855,11540,627,4,159.96,39.99', u'28856,11540,1014,2,99.96,49.98']
11546
[u'28867,11546,1004,1,399.98,399.98']
sorted order item group by key
28863,11542,191,5,499.95,99.99
28861,11542,365,4,239.96,59.99
28864,11542,1014,3,149.94,49.98
28862,11542,627,2,79.98,39.99
28860,11542,1014,1,49.98,49.98
28855,11540,627,4,159.96,39.99
28856,11540,1014,2,99.96,49.98
28867,11546,1004,1,399.98,399.98
28865,11544,1073,1,199.99,199.99
28866,11544,1014,4,199.92,49.98


### reduceBYKey

In [14]:
orderItems = sc.textFile("retail_db/retail_db/order_items")
orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]), float(oi.split(",")[4])))

#for i in orderItemsMap.take(5):print i
print "Sum of all order items by order id"    
orderItemsReduceByKey = orderItemsMap.reduceByKey(lambda x,y: x+y)    

for i in orderItemsReduceByKey.take(3):print i

print "*"*30      
print "Sum of all order items by using add operator"    
from operator import add
orderItemsReduceByKey = orderItemsMap.reduceByKey(add)
for i in orderItemsReduceByKey.take(3):print i

print "*"*30    
print "Min value per order id"    
orderItemsReduceByKey = orderItemsMap.reduceByKey(lambda x,y : x if( x < y) else y)
for i in orderItemsReduceByKey.take(3):print i
    


Max value per order id
(2, 579.98)
(4, 699.85)
(8, 729.8399999999999)
******************************
Max value per order id using add operator
(2, 579.98)
(4, 699.85)
(8, 729.8399999999999)
******************************
Min value per order id
(2, 129.99)
(4, 49.98)
(8, 50.0)
******************************
(1, u'1,1,957,1,299.98,299.98')
(2, u'2,2,1073,1,199.99,199.99')
(2, u'3,2,502,5,250.0,50.0')


### aggregateByKey


In [28]:
#Get order items count & total revenue per order

#As we already formated data using map like (order id, subtotal) or we can name it (oi,subtotal)
#                                           ( 1      ,  299.98)

# Our desired output is like (orderId,( sum[subtotal order], count order items in a order))
# (2,(579.98, 3)) name it ( oi,  ( total , count) )

# aggregateByKey(
#               (initial val of total , initial val of count ) ,
#               (lamdba ('total' last val , 'count' last val) , subtotal ),
#               (lambda (('total' last val , 'count' last val) , subtotal ), (('total' last val , 'count' last val) , subtotal ) 
#

# aggregateByKey( initialize value, intermidiator logic , combinor logic )

# Input
# (2,300)
# (2,500)
# (2,100)
# (2,5)

# Requried Output(2, (950,4) )
# initiaze (0.0 , 0 ) (0.0 initial value of total, 0 initial value of count)

# In aggregate intermiator we have to define logic to add  initiaze + given input
# (2,300)
# (2,500)
# --------
# (800,2) name it as s
# --------

orderItems = sc.textFile("retail_db/retail_db/order_items")
orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]), float(oi.split(",")[4])))
orderItemsAggregate = orderItemsMap.aggregateByKey( \
                             (0.0,0), \
                             (lambda x,y : (x[0] + y ,x[1] + 1 ) ),\
                             (lambda a,b : ( (a[0] + b[0]), a[1],b[1] ) ) \
                            )
for i in orderItemsAggregate.take(5): print i
#help(orderItemsMap.aggregateByKey)

# (2,100)
# (2,50)
# -----
# (150,3) name it as q
# --------
# In aggregate combinor define logic to add results of intermiator outputs belongs to one same key
#  s + q 
# (800,2)
# (150,3)
# (950,5)





(2, (579.98, 3))
(4, (699.85, 4))
(8, (729.8399999999999, 4))
(10, (651.9200000000001, 5))
(12, (1299.8700000000001, 5))


## Sorting 
#### Get products price in desecding order

In [43]:
products = sc.textFile("retail_db/retail_db/products")
#for i in products.take(2): print i

#print products.count()   

# filter invalid records
filteredProducts = products.filter(lambda p: p.split(",")[4] != "")

#print filteredProducts.count()
#for i in filteredProducts.take(2): print i    
    
productsMap = filteredProducts.map(lambda p: (float(p.split(",")[4]), p))
#for i in productsMap.take(2): print i


sortedProducts =   productsMap.sortByKey(False)
#for i in sortedProducts.take(3): print i
    
sortedProductsMap = sortedProducts.map(\
                                       lambda x : x[1])

for i in sortedProductsMap.take(3): print i

208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Elliptical
66,4,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill
199,10,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill


### Sort data using combined key
### by product category & then product price desending

In [68]:
products = sc.textFile("retail_db/retail_db/products")
#for i in products.take(1): print i

# filter invalid records
filteredProducts = products.filter(lambda p: p.split(",")[4] != "")

# product price ascending
productsMapAscending = filteredProducts.map(lambda p: ( \
                                            (int(p.split(",")[1]) , float(p.split(",")[4] ))  , \
                                             p ) \
                                  ) 
                 
sortedProductByCategory = productsMapAscending.sortByKey()    
#for i in sortedProductByCategory.take(1):print i
    
productsMapDesending = filteredProducts.map(lambda p: ( \
                                            (int(p.split(",")[1]) , -float(p.split(",")[4] ))  , \
                                             p ) \
                                  )     

sortedProductByCategory = productsMapDesending.sortByKey().map(lambda p: p[1])      
for i in sortedProductByCategory.take(10):print i
    
    
    
# (categoryId ,price) p)


#help(productsMap.reduceByKey)    
    

16,2,Riddell Youth 360 Custom Football Helmet,,299.99,http://images.acmesports.sports/Riddell+Youth+360+Custom+Football+Helmet
11,2,Fitness Gear 300 lb Olympic Weight Set,,209.99,http://images.acmesports.sports/Fitness+Gear+300+lb+Olympic+Weight+Set
5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet
14,2,Quik Shade Summit SX170 10 FT. x 10 FT. Canop,,199.99,http://images.acmesports.sports/Quik+Shade+Summit+SX170+10+FT.+x+10+FT.+Canopy
12,2,Under Armour Men's Highlight MC Alter Ego Fla,,139.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Alter+Ego+Flash+Football...
23,2,Under Armour Men's Highlight MC Alter Ego Hul,,139.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Alter+Ego+Hulk+Football...
6,2,Jordan Men's VI Retro TD Football Cleat,,134.99,http://images.acmesports.sports/Jordan+Men%27s+VI+Retro+TD+Football+Cleat
2,2,Under Armour Men's Highlight MC F